<a href="https://colab.research.google.com/github/naina843/StockTracker/blob/main/MoneySentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install unstructured libmagic python-magic python-magic-bin
!pip install bs4
!pip install -qU langchain openai
!pip install -U langchain-community
!pip install langchain unstructured nltk
!pip install faiss-cpu
!pip install sentence_transformers
!pip install tiktoken
!pip install groq
!pip install -qU langchain-groq
!pip install langchain
!pip install langchain-huggingface
!pip install --upgrade nltk
import nltk
nltk.download('vader_lexicon')
nltk.download('punkt')
!pip install spacy
!python -m spacy download en_core_web_sm

  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement python-magic-bin (from versions: none)
ERROR: No matching distribution found for python-magic-bin
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.5/474.5 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
  Using cached unstructured-0.16.25-py3-none-any.whl.metadata (24 kB)
  Using cached python_magic-0.4.27-py2.py3-none-any.whl.metadata (5.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 23.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 126.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pickle
import os
import nltk
import langchain
import groq
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.chains.question_answering import load_qa_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from  langchain.document_loaders import UnstructuredURLLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [4]:
article_title=[]
article_urls=[]

In [5]:
url = "https://www.moneycontrol.com/news/business/stocks/"

response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
print('MoneyControl')
for article in soup.find_all('li', class_='clearfix'):
    title = article.find('a')['title']
    link = article.find('a')['href']
    article_title.append(title)
    article_urls.append(link)
    print("Title: ", title)
    print("Link: ", link)
print('-'*50)
print('CNBC')
url = "https://www.cnbctv18.com/stocks/"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
articles = soup.find_all('a', class_='jsx-95506e352219bddb story-media')
for article in articles:
    title = article['title'].strip()
    link = urljoin(url, article['href'])
    article_title.append(title)
    article_urls.append(link)
    print("Title: ", title)
    print("Link: ", link)

MoneyControl
Title:  SPML Infra shares rally 5% on bagging Rs 618 crore irrigation project in Jharkhand
Link:  https://www.moneycontrol.com/news/business/stocks/spml-infra-shares-rally-5-on-bagging-rs-618-crore-irrigation-project-in-jharkhand-12960396.html
Title:  Sensex up 200 pts, Nifty at 22,600 as metal, FMCG stocks gain; oil & gas, realty drag
Link:  https://www.moneycontrol.com/news/business/markets/sensex-up-200-pts-nifty-at-22-600-as-metal-fmcg-stocks-gain-oil-gas-realty-drag-12960446.html
Title:  Metal shares surge to extend gains for a fifth session on Jefferies' bullish call
Link:  https://www.moneycontrol.com/news/business/markets/metal-shares-surge-to-extend-gains-for-a-fifth-session-on-jefferies-bullish-call-12960400.html
Title:  Buy IndusInd Bank; target of Rs 1100: Motilal Oswal
Link:  https://www.moneycontrol.com/news/business/stocks/buy-indusind-bank-target-of-rs-1100-motilal-oswal-12960466.html
Title:  Chart of the Day: A historical perspective on market declines and

In [6]:
article_title

['SPML Infra shares rally 5% on bagging Rs 618 crore irrigation project in Jharkhand',
 'Sensex up 200 pts, Nifty at 22,600 as metal, FMCG stocks gain; oil & gas, realty drag',
 "Metal shares surge to extend gains for a fifth session on Jefferies' bullish call",
 'Buy IndusInd Bank; target of Rs 1100: Motilal Oswal',
 'Chart of the Day: A historical perspective on market declines and recovery',
 'Trent shares fall over 2% as Kotak Institutional Equities cuts estimate, cites revenue pressure',
 'Aavas Financiers shares surge 9% amid high trading volumes',
 'Accumulate VA Tech Wabag; target of Rs 1571: Geojit Financial Services',
 'Broader markets BSE mid and smallcap indices flat as Trump tariffs weigh on sentiment',
 'Buy Oil and Natural Gas Corporation; target of Rs 261: Geojit Financial Services',
 'Reduce IndusInd Bank; target of Rs 850: ICICI Securities',
 'Motilal Oswal sees strong growth potential in Bharti Hexacom on wireless and broadband expansion',
 'Hold Inventurus Knowledge

In [7]:
article_urls

['https://www.moneycontrol.com/news/business/stocks/spml-infra-shares-rally-5-on-bagging-rs-618-crore-irrigation-project-in-jharkhand-12960396.html',
 'https://www.moneycontrol.com/news/business/markets/sensex-up-200-pts-nifty-at-22-600-as-metal-fmcg-stocks-gain-oil-gas-realty-drag-12960446.html',
 'https://www.moneycontrol.com/news/business/markets/metal-shares-surge-to-extend-gains-for-a-fifth-session-on-jefferies-bullish-call-12960400.html',
 'https://www.moneycontrol.com/news/business/stocks/buy-indusind-bank-target-of-rs-1100-motilal-oswal-12960466.html',
 'https://www.moneycontrol.com/news/opinion/chart-of-the-day-a-historical-perspective-on-market-declines-and-recovery-12960452.html',
 'https://www.moneycontrol.com/news/business/markets/trent-shares-fall-over-2-as-kotak-institutional-equities-cuts-estimate-cites-revenue-pressure-12960422.html',
 'https://www.moneycontrol.com/news/business/stocks/aavas-financiers-shares-surge-9-amid-high-trading-volumes-12960330.html',
 'https://

In [14]:
os.environ['GROQ_API_KEY']='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'

In [15]:
# try:
#     nltk.data.find('tokenizers/punkt')
# except LookupError:
#     nltk.download('punkt')

# data=[]
# for i in range(len(article_urls)):
#     url = article_urls[i]
#     loader = UnstructuredURLLoader(urls=[url])
#     try:
#       documents = loader.load()
#       for doc in documents:
#         data.append(doc)
#         print(doc.page_content)
#     except Exception as e:
#       print(f"Error loading URL: {e}")

In [16]:
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

loader = UnstructuredURLLoader(urls=['https://www.moneycontrol.com/news/business/stocks/buy-mahindra-and-mahindra-financial-services-target-of-rs-380-axis-capital-12827693.html'])
try:
  data = loader.load()
  print('Loaded')
except Exception as e:
  print(f"Error loading URL: {e}")

Loaded


In [20]:
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.9,
    max_tokens=500,
)


In [21]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs=text_splitter.split_documents(data)
len(docs)

7

In [22]:
embeddings=HuggingFaceEmbeddings()
vector_index=FAISS.from_documents(docs,embeddings)

In [23]:
chain=RetrievalQAWithSourcesChain.from_llm(llm=llm,retriever=vector_index.as_retriever())
chain

RetrievalQAWithSourcesChain(verbose=False, combine_documents_chain=MapReduceDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7ea2b272fc90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7ea2b6c1bc90>, model_name='llama-3.3-70b-versatile', temperature=0.9, model_kwargs={}, groq_api_key=SecretStr('**********'), max_tokens=500), output_parser=StrOutputParser(), llm_kwargs={}), reduce_documents_chain=ReduceDocumentsChain(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variabl

In [24]:
query="Give a small summary of the article"
langchain.debug=True
response=chain({"question":query},return_only_outputs=True)

<ipython-input-24-39c1987d3294>:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=chain({"question":query},return_only_outputs=True)


[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "Give a small summary of the article"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "Community\n\nStock Markets\n\nNetwork 18 Sites\n\nNews18 Firstpost CNBC TV18 News18 Hindi Cricketnext Overdrive Topper Learning\n\nQuick Links\n\nAbout Us Contact Us Advisory Alert Advertise with Us SupportDisclaimer Privacy Policy Cookie Policy Terms & Conditions Financial Terms (Glossary) Sitemap Investors\n\nDownload MC Apps:\n\nCopyright © Network18 Media & Investments Limited. All rights reserved. Reproduction of news articles, photos, videos or any other content in whole or in part in any form or medium without express written permission of moneycontrol.c

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1892 > 1024). Running this sequence through the model will result in indexing errors


[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Give a small summary of the article",
  "summaries": "Content: There is no relevant text to summarize as the provided portion of the document appears to be a website's footer or sidebar, containing links, copyright information, and promotional content, but no article text.\nSource: https://www.moneycontrol.com/news/business/stocks/buy-mahindra-and-mahindra-financial-services-target-of-rs-380-axis-capital-12827693.html\n\nContent: Axis Capital is bullish on Mahindra and Mahindra Financial Services has recommended buy rating on the stock with a target price of Rs 380 in its research report dated September 19, 2024.\nSource: https://www.moneycontrol.com/news/business/stocks/buy-mahindra-and-mahindra-financial-services-target-of-rs-380-axis-capital-12827693.html\n\nContent: There is no article provided in the given text. The text appears to be a

In [25]:
answer = response["answer"]
answer = answer.replace("\n", "")
print(answer)


The article discusses Axis Capital's recommendation to buy Mahindra and Mahindra Financial Services with a target price of Rs 380.


In [26]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score , classification_report

analyzer=SentimentIntensityAnalyzer()

In [28]:
df=pd.read_csv('all-data.csv',encoding='latin')
df.columns=['class','text']
df['text'] = df['text'].astype(str)
df

,class,text
0,neutral,Technopolis plans to develop in stages an area...
1,negative,The international electronic industry company ...
2,positive,With the new production plant the company woul...
3,positive,According to the company 's updated strategy f...
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...
...,...,...
4840,negative,LONDON MarketWatch -- Share prices ended lower...
4841,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4842,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4843,negative,Net sales of the Paper segment decreased to EU...


In [29]:
df.duplicated().sum()

6

In [30]:
df.drop_duplicates(inplace=True)

In [31]:
df.isnull().sum()

,0
class,0
text,0


In [32]:
df['class'].value_counts()
df=df[~(df['class']=='neutral')]
df['class'].value_counts()

,count
class,
positive,1363
negative,604


In [33]:
X_train,X_test,y_train,y_test=train_test_split(df['text'].values,df['class'].values,random_state=42,test_size=0.2)

In [34]:
analyzer.polarity_scores(df['text'].values[4])

{'neg': 0.064, 'neu': 0.856, 'pos': 0.08, 'compound': 0.1531}

In [35]:
df['vader_pred']=df['text'].apply(lambda x : analyzer.polarity_scores(x)['compound'])

<ipython-input-35-2c5b8a5e7dce>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['vader_pred']=df['text'].apply(lambda x : analyzer.polarity_scores(x)['compound'])


In [36]:
df['pred_class']=df['vader_pred'].apply(lambda x: 'negative' if x <0 else 'positive' )

<ipython-input-36-1ea230deb47b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pred_class']=df['vader_pred'].apply(lambda x: 'negative' if x <0 else 'positive' )


In [37]:
df

,class,text,vader_pred,pred_class
1,negative,The international electronic industry company ...,0.0000,positive
2,positive,With the new production plant the company woul...,0.8555,positive
3,positive,According to the company 's updated strategy f...,0.6705,positive
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,0.3485,positive
5,positive,"For the last quarter of 2010 , Componenta 's n...",0.1531,positive
...,...,...,...,...
4839,negative,HELSINKI Thomson Financial - Shares in Cargote...,0.6597,positive
4840,negative,LONDON MarketWatch -- Share prices ended lower...,-0.7269,negative
4842,negative,Operating profit fell to EUR 35.4 mn from EUR ...,0.7430,positive
4843,negative,Net sales of the Paper segment decreased to EU...,0.4404,positive


In [38]:
print(classification_report(df['class'],df['pred_class']))

              precision    recall  f1-score   support

    negative       0.71      0.31      0.43       604
    positive       0.76      0.94      0.84      1363

    accuracy                           0.75      1967
   macro avg       0.73      0.63      0.64      1967
weighted avg       0.74      0.75      0.71      1967



In [39]:
vader_score = analyzer.polarity_scores(answer)
predicted_class = 'negative' if vader_score['compound'] < 0 else 'positive'
print(f"VADER Sentiment Scores: {vader_score}")
print(f"Predicted Sentiment Class: {predicted_class}")

VADER Sentiment Scores: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
Predicted Sentiment Class: positive


In [41]:
nse_stocks= pd.read_csv('stocks&symb - Sheet1.csv')
nse_stocks

,NSE,Name,Symbol
0,NSE: 20MICRONS,20 Microns Limited,20MICRONS
1,NSE: 21STCENMGM,21st Century Management Services Limited,21STCENMGM
2,NSE: 3IINFOTECH,3i Infotech Limited,3IINFOTECH
3,NaN,3M India Limited,#REF!
4,NSE: 3PLAND,3P Land Holdings Limited,3PLAND
...,...,...,...
1808,NSE: ZOTA,Zota Health Care Limited,ZOTA
1809,NSE: ZUARI,Zuari Agro Chemicals Limited,ZUARI
1810,NSE: ZUARIGLOB,Zuari Global Limited,ZUARIGLOB
1811,NSE: ZYDUSWELL,Zydus Wellness Limited,ZYDUSWELL


In [42]:
symbols= nse_stocks[['Name','Symbol']].copy()
# symbols['Name'] = symbols['Name'].astype(pd.StringDtype())
# symbols['Symbol'] = symbols['Symbol'].astype(pd.StringDtype())
symbols

,Name,Symbol
0,20 Microns Limited,20MICRONS
1,21st Century Management Services Limited,21STCENMGM
2,3i Infotech Limited,3IINFOTECH
3,3M India Limited,#REF!
4,3P Land Holdings Limited,3PLAND
...,...,...
1808,Zota Health Care Limited,ZOTA
1809,Zuari Agro Chemicals Limited,ZUARI
1810,Zuari Global Limited,ZUARIGLOB
1811,Zydus Wellness Limited,ZYDUSWELL


In [43]:
query="Give me the name of main company on which doc is based"
langchain.debug=True
comp=chain({"question":query},return_only_outputs=True)

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "Give me the name of main company on which doc is based"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "Mahindra & Mahindra - 23092024 - axis\n\nBroker Research\n\nfirst published: Sep 23, 2024 12:51 pm\n\nDiscover the latest Business News, Budget 2025 News, Sensex, and Nifty updates. Obtain Personal Finance insights, tax queries, and expert opinions on Moneycontrol or download the Moneycontrol App to stay updated!\n\nAdvertisement\n\nRemove Ad\n\nTrending news\n\nTop YouTuber MrBig has an $85 million empire, says others wouldn't want his life: 'Don’t have free will'\n\nKiran Mazumdar-Shaw, Mohandas Pai support Bengaluru protests for better roa

In [44]:
company_str = comp["answer"]
company_str = company_str.replace("\n", "")
print(company_str)

The main company on which the document is based is Mahindra and Mahindra Financial Services, which is a part of the Mahindra & Mahindra group.


In [45]:
import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp(company_str)
stock_names = [ent.text for ent in doc.ents if ent.label_ == "ORG"]
print(stock_names)

['Mahindra and Mahindra Financial Services', 'the Mahindra & Mahindra']


In [47]:
variations = []
for name in stock_names:
    if '&' in name:
        variations.append(name.replace('&', 'and'))
    elif 'and' in name:
        variations.append(name.replace('and', '&'))
stock_names.extend(variations)
print(stock_names)

['Mahindra and Mahindra Financial Services', 'the Mahindra & Mahindra', 'Mahindra & Mahindra Financial Services', 'the Mahindra and Mahindra', 'Mahindra & Mahindra Financial Services', 'the Mahindra and Mahindra', 'Mahindra and Mahindra Financial Services', 'the Mahindra & Mahindra']


In [48]:
matching_symbols = symbols[symbols['Name'].apply(lambda x: any(name in x for name in stock_names))]['Symbol'].tolist()
print(matching_symbols)

['M&MFIN']
